In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('CKD.csv')
dataset=pd.get_dummies(dataset,drop_first=True).astype(int)

In [3]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2,76,3,0,148,57,3,137,4,12,...,0,0,0,0,0,0,1,1,0,1
1,3,76,2,0,148,22,0,137,4,10,...,1,0,0,0,0,0,1,0,0,1
2,4,76,1,0,99,23,0,138,4,12,...,1,0,0,0,0,0,1,0,0,1
3,5,76,1,0,148,16,0,138,3,8,...,1,0,0,0,0,0,1,0,1,1
4,5,50,0,0,148,25,0,137,4,11,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51,70,0,0,219,36,1,139,3,12,...,1,0,0,0,0,0,1,0,0,1
395,51,70,0,2,220,68,2,137,4,8,...,1,0,0,1,1,0,1,0,1,1
396,51,70,3,0,110,115,6,134,2,9,...,1,0,0,1,1,0,0,0,0,1
397,51,90,0,0,207,80,6,142,5,8,...,1,0,0,1,1,0,1,0,1,1


In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [6]:
dep=dataset[['classification_yes']]

In [7]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(indep,dep,test_size=1/3,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train-sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [9]:
from sklearn.svm import SVC

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid={'C':[1000,2000,3000],'kernel':['linear', 'poly']}
grid=GridSearchCV(SVC(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [1000, 2000, 3000], 'kernel': ['linear', 'poly']},
             verbose=3)

In [11]:
grid_prediction=grid.predict(x_test)

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [14]:
re=grid.cv_results_
print(grid.best_params_)

{'C': 3000, 'kernel': 'linear'}


In [16]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print(cm)

[[ 0 51]
 [ 0 82]]


In [18]:
from sklearn.metrics import classification_report as clf
clf_report=clf(y_test,grid_prediction)

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.62      1.00      0.76        82

    accuracy                           0.62       133
   macro avg       0.31      0.50      0.38       133
weighted avg       0.38      0.62      0.47       133



In [21]:
from sklearn.metrics import roc_auc_score as r
r(y_test,grid_prediction)

np.float64(0.5)